OData API

https://oda.ft.dk/Home/OdataQuery

https://www.odata.org/

https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx

https://www.odata.org/documentation/odata-version-3-0/url-conventions/

https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/

Source root Link: "https://oda.ft.dk/api/"

ftp://oda.ft.dk/ODAXML/Referat/samling/


## Folketinget data

The data related to the meetings in Folketinget is collected following the guide on Folketinget's homepage:
https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx.
Transcripts from all meetings going back to 2009 are available for download in xml format using a login described in the guide and some specific ftp software. Because it is interesting to compare topics from Twitter with topics in Folketinget, only the transcripts from the same period as the tweets are used for further analysis. 71% of the available tweets are posted in 2018 and after.

The years in Folketinget do not follow the calender year, as they start and end the first Tuesday of October. So, the meetings from 2018 are actually from October 2017 to October 2018. The data from 2017 are kept for the purpose of having slightly more data and because it is assumed that topics vary little enought for it to still be relevant.

In total 284 meeting stranscripts amounting to 256.6Mb of data has been downloaded and parsed for analysis.

It is not stated how the transcripts are made, however, they are proofread before publication. 


In [ ]:
#pip install bs4 lxml

In [1]:
import requests
import urllib.request
import os
import io
import json

import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import lxml

import xml.etree.ElementTree as et 
from xml.dom import minidom

import re

import dateutil
import datetime as dt
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *

In [2]:
path = os.getcwd()

In [3]:
path = path+'/samling/'

# Parsing XML

In [4]:
i = 1
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if filepath.endswith(".xml") and i == 1:
            xmlTree = et.parse(filepath)

            
elemList = []

for elem in xmlTree.iter():
    elemList.append(elem.tag)

# now I remove duplicities - by convertion to set and back to list
#elemList = list(set(elemList))

# Just printing out the result
from collections import Counter
Counter(elemList)

Counter({'Dokument': 1,
         'MetaMeeting': 1,
         'ParliamentarySession': 1,
         'ParliamentaryGroup': 1,
         'MeetingNumber': 1,
         'DateOfSitting': 1,
         'Location': 1,
         'EdixiDocLocation': 1,
         'AudioFileFolder': 1,
         'TitelGruppe': 1,
         'Titel': 1,
         'Linea': 126,
         'Char': 138,
         'UnderTitel': 1,
         'DagsordenPlan': 1,
         'Rubrica': 10,
         'PunktTekst': 8,
         'Exitus': 72,
         'DagsordenPunkt': 5,
         'MetaFTAgendaItem': 5,
         'ItemNo': 5,
         'FTCase': 5,
         'FTCaseNumber': 5,
         'FTCaseType': 5,
         'FTCaseStage': 5,
         'ShortTitle': 5,
         'Aktivitet': 10,
         'Tale': 12,
         'Taler': 12,
         'MetaSpeakerMP': 12,
         'OratorFirstName': 12,
         'OratorLastName': 12,
         'GroupNameShort': 12,
         'OratorRole': 12,
         'TalerTitel': 12,
         'TaleSegment': 12,
         'MetaSpeechSegme

In [98]:
i = 1
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if filepath.endswith("clean.xml") and i == 1:
            xmlTree = et.parse(filepath)
            i += 1
filepath

'/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M109_helemoedet.xml'

In [65]:
elemList = []

for elem in xmlTree.iter():
    elemList.append(elem.tag)

# Just printing out the result
from collections import Counter
Counter(elemList)

Counter({'Dokument': 1,
         'MetaMeeting': 1,
         'ParliamentarySession': 1,
         'ParliamentaryGroup': 1,
         'MeetingNumber': 1,
         'DateOfSitting': 1,
         'Location': 1,
         'EdixiDocLocation': 1,
         'AudioFileFolder': 1,
         'TitelGruppe': 1,
         'Titel': 1,
         'Linea': 2433,
         'Char': 2968,
         'UnderTitel': 1,
         'DagsordenPlan': 1,
         'Rubrica': 4,
         'PunktTekst': 60,
         'Exitus': 885,
         'DagsordenPunkt': 3,
         'MetaFTAgendaItem': 3,
         'ItemNo': 3,
         'FTCase': 30,
         'FTCaseNumber': 30,
         'FTCaseType': 30,
         'FTCaseStage': 30,
         'ShortTitle': 31,
         'Aktivitet': 5,
         'Tale': 399,
         'Taler': 399,
         'MetaSpeakerMP': 399,
         'OratorFirstName': 399,
         'OratorLastName': 399,
         'GroupNameShort': 399,
         'OratorRole': 399,
         'TalerTitel': 399,
         'TaleSegment': 399,
         

In [68]:
elemList

['Dokument',
 'MetaMeeting',
 'ParliamentarySession',
 'ParliamentaryGroup',
 'MeetingNumber',
 'DateOfSitting',
 'Location',
 'EdixiDocLocation',
 'AudioFileFolder',
 'TitelGruppe',
 'Titel',
 'Linea',
 'Char',
 'UnderTitel',
 'Linea',
 'Char',
 'DagsordenPlan',
 'Rubrica',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'PunktTekst',
 'Exitus',
 'Linea',
 'Char',
 'Exitus',
 'Linea',
 'Char',
 'Char',


In [85]:
root.tag
root.attrib

{'edixistatus': 'Foreløbig'}

In [86]:
for child in root:
    print(child.tag, child.attrib)

MetaMeeting {}
TitelGruppe {}
DagsordenPlan {}
DagsordenPunkt {}
DagsordenPunkt {}
DagsordenPunkt {}
DagsordenPunkt {}
DagsordenPunkt {}


In [88]:
print(et.tostring(root, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<Dokument edixistatus="Foreløbig"><MetaMeeting><ParliamentarySession>20191</ParliamentarySession><ParliamentaryGroup>Folketinget</ParliamentaryGroup><MeetingNumber>109</MeetingNumber><DateOfSitting>2020-05-14T10:00:00</DateOfSitting><Location>Folketingssalen</Location><EdixiDocLocation>W:\20191\M109\</EdixiDocLocation><AudioFileFolder>X:\Arbor\FT2019\109\SALEN\</AudioFileFolder></MetaMeeting><TitelGruppe><Titel><Linea><Char>109. møde</Char></Linea></Titel><UnderTitel><Linea><Char>Torsdag den 14. maj 2020 kl. 10.00</Char></Linea></UnderTitel></TitelGruppe><DagsordenPlan><Rubrica><Linea><Char>Dagsorden</Char></Linea></Rubrica><PunktTekst><Exitus><Linea><Char>1) 2. behandling af lovforslag nr. L 107:</Char></Linea></Exitus><Exitus><Linea><Char>Forslag til lov om ændring af SU-loven, lov om statens voksenuddannelsesstøtte, lov om befordringsrabat til studerende ved videregående uddannelser og lov om befordringsrabat til uddannelsessøgende i ungdomsudda

In [96]:
for movie in root.iter('ShortTitle'):
    print(movie.text)

Punkt 0
2. behandling af L 107: Om SU til adgangskurser på institutioner for erhvervsrettet uddannelse m.v.
2. behandling af L 124: Om håndhævelse af P2B-forordningen.
2. behandling af L 181: Om ændret tilbagebetalingsmodel i forbindelse med udvidet dækningsområde for Rejsegarantifond
Punkt 0


In [5]:
for speech in root.iter('MeetingNumber'):
    print(speech.text)

NameError: name 'root' is not defined

In [48]:
starttid_vec = []
sluttid_vec=[]
navn_vec = []
efternavn_vec = []
tekst_vec = []
parti_vec = []
rolle_vec = []
meetingID_vec = []
itemNo_vec = []

# Go through all the folders and files containing transcripts
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if filepath.endswith(".xml"):
            tree = et.parse(filepath)
            root = tree.getroot() 
            
            print(filepath)
            
            
            for meeting in root.iter('MeetingNumber'):
                meetingID = meeting.text

                for ItemNo in root.iter('ItemNo'):
                    
                # Do this if the speaker has an associated party, i.e. not a minister or the chairman
                    for speech in root.iter('Tale'):

                        tags=[node.tag for node in speech.iter()]
                        if('TekstGruppe' in tags):
                            if('GroupNameShort' in tags):
                                for node in speech.iter():
                                    if(node.tag == 'StartDateTime'):
                                        starttid_vec.append(node.text)
                                    elif(node.tag == 'OratorFirstName'):
                                        navn_vec.append(node.text)
                                    elif(node.tag == 'OratorLastName'):
                                        efternavn_vec.append(node.text)
                                        #print('Meeting ID: ', meetingID, 'Item No: ', ItemNo.text, 'Speaker: ', node.text)
                                    elif(node.tag == 'OratorRole'):
                                        rolle_vec.append(node.text)
                                    elif(node.tag=='TekstGruppe'):
                                        # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                        # Concate all text within 'TekstGruppe'.
                                        tekstgruppe = ''
                                        for k in node.iter():
                                            if k.tag=="Char":
                                                tekstgruppe+=' '+k.text
                                        tekst_vec.append(tekstgruppe)
                                    elif(node.tag=='GroupNameShort'):
                                        parti_vec.append(node.text)

                                        meetingID_vec.append(meetingID)
                                        itemNo_vec.append(ItemNo.text)


                                    # If there are more text than speaker names, remove the excess.
                                    # We will loose a few pieces of text but only around 1%
                                    if len(tekst_vec)>len(navn_vec):
                                        tekst_vec.pop()
                                    if len(starttid_vec)>len(navn_vec):
                                        starttid_vec.pop()

                            # Do this if the speaker is either a minister or the chairman
                            else:
                                parti_vec.append('Ukendt')
                                for node in speech.iter():
                                    if(node.tag == 'StartDateTime'):
                                        starttid_vec.append(node.text)
                                    elif(node.tag == 'OratorFirstName'):
                                        navn_vec.append(node.text)
                                    elif(node.tag == 'OratorLastName'):
                                        efternavn_vec.append(node.text)
                                    elif(node.tag == 'OratorRole'):
                                        rolle_vec.append(node.text)
                                    elif(node.tag=='TekstGruppe'):
                                        # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                        # Concate all text within 'TekstGruppe'.
                                        tekstgruppe = ''
                                        for k in node.iter():
                                            if k.tag=="Char":
                                                tekstgruppe+=' '+k.text
                                        tekst_vec.append(tekstgruppe)

                                        meetingID_vec.append(meetingID)
                                        itemNo_vec.append(ItemNo.text)

                                    # If there are more text than speaker names, remove the excess.
                                    # We will loose a few pieces of text but only around 1%
                                    if len(tekst_vec)>len(navn_vec):
                                        tekst_vec.pop()
                                    if len(starttid_vec)>len(navn_vec):
                                        starttid_vec.pop()
                    


# Add everuthing to a dictionary and transform it into a Pandas dataframe
dictionary= {'MeetingID':meetingID_vec, 'ItemNo':itemNo_vec, 'StartDateTime':starttid_vec,'OratorFirstName':navn_vec,'OratorLastName':efternavn_vec,
                 'GroupNameShort':parti_vec,'OratorRole':rolle_vec ,'TekstGruppe':tekst_vec} # 'EndDateTime':sluttid_vec,
df = pd.DataFrame(dictionary)

print('In total, there are {} lines of text in the meetings.'.format(len(df)))
#df.to_csv(path[0:26]+'csv')

/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M66_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M73_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M17_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M91_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M84_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M6_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M74_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M61_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M9_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M10_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politi

/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M48_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M52_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M47_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M36_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20181/20181_M23_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20201/20201_M2_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20201/20201_M27_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20201/20201_M5_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20201/20201_M28_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20201/20201_M20_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politi

/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M29_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M72_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M94_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M10_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M113_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M13_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M60_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M75_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M117_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M129_helemoedet 2.xml
/Users/frederikkromannhansen/Docu

/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M125_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M130_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M138_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M91_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M77_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M49_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M89_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M12_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M128_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M149_helemoedet.xml
/Users/frederikkromannhansen/Document

/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M53_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M131_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M7_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M108_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M22_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M37_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M8_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M14_helemoedet 2.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M115_helemoedet.xml
/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/20191/20191_M100_helemoedet.xml
/Users/frederikkromannhansen/Documents/Git

In [30]:
df.head(10)

,MeetingID,ItemNo,StartDateTime,OratorFirstName,OratorLastName,GroupNameShort,OratorRole,TekstGruppe
0,66,0,2019-02-27T13:00:08,Pia,Kjærsgaard,DF,formand,Mødet er åbnet.
1,66,0,2019-02-27T13:01:34,Pia,Kjærsgaard,DF,formand,Det første spørgsmål er til justitsministeren...
2,66,0,2019-02-27T13:01:43,Pia,Kjærsgaard,DF,formand,Værsgo for at oplæse spørgsmålet.
3,66,0,2019-02-27T13:01:45,Christian,Langballe,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...
4,66,0,2019-02-27T13:02:05,Pia,Kjærsgaard,DF,formand,"Værsgo, ministeren."
5,66,0,2019-02-27T13:02:06,Søren Pape,Poulsen,None,minister,Tak for det. Og tak for spørgsmålet. Fremmedk...
6,66,0,2019-02-27T13:03:39,Pia,Kjærsgaard,DF,formand,"Tak. Værsgo, hr. Christian Langballe."
7,66,0,2019-02-27T13:03:40,Christian,Langballe,DF,medlem,Det står tilbage at få besvaret nogle ting i ...
8,66,0,2019-02-27T13:04:53,Pia,Kjærsgaard,DF,formand,Tak. Og værsgo.
9,66,0,2019-02-27T13:04:55,Søren Pape,Poulsen,None,minister,"Tak. Jeg skal prøve, så godt jeg kan – det va..."


In [31]:
len(meetingID_vec)

1714830

In [49]:
### Cleaning the data ###

# The chariman (formand) does not add value to the context as he or she only moderates the debate
# by passing on the word from person to person.
# Also some meta data is contained regarding when the meeting ended (MødeSlut). This is also irrelevant.
df=df[df['OratorRole']!='formand']
df=df[df['OratorRole']!='MødeSlut']
df['OratorRole'].dropna(axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)

# Generating a column of the full name of the speakers
df['FullName'] = df['OratorFirstName']+' '+df['OratorLastName']

# Making start time into datetime64 format
df['StartDateTime'] = df['StartDateTime'].apply(lambda x: dateutil.parser.parse(x))

df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))
df['ItemNo']=df['ItemNo'].apply(lambda x: int(x))

In [53]:
what = df.groupby(['MeetingID','ItemNo'])['TekstGruppe'].apply(lambda x : ' '.join(x))

In [60]:
what.head(20)

MeetingID  ItemNo
1          0          Deres Majestæt, Deres Kongelige Højheder. I e...
           1          Deres Majestæt, Deres Kongelige Højheder. I e...
           2          Deres Majestæt, Deres Kongelige Højheder. I e...
           3          Deres Majestæt, Deres Kongelige Højheder. I e...
           4          Deres Majestæt, Deres Kongelige Højheder. I e...
2          0          Tak for ordet, formand. Lovforslaget, som vi ...
           1          Tak for ordet, formand. Lovforslaget, som vi ...
           2          Tak for ordet, formand. Lovforslaget, som vi ...
           3          Tak for ordet, formand. Lovforslaget, som vi ...
           4          Mødet er åbnet, og jeg beder medlemmerne find...
           5          Mødet er åbnet, og jeg beder medlemmerne find...
3          0          Tid er en underlig størrelse. Jeg er f.eks. s...
           1          Tid er en underlig størrelse. Jeg er f.eks. s...
           2          Tid er en underlig størrelse. Jeg er 

In [78]:
what[1][1]

' Deres Majestæt, Deres Kongelige Højheder. I en tid, hvor demokratiet og folkestyrets institutioner i mange lande er under pres, er der al mulig grund til at højtideligholde folkestyrets festdag her i det danske Folketing. Den første tirsdag i oktober er en festdag, hvor folketingsåret efter gudstjenesten indledes med, at Folketinget i nærværelse af Hendes Majestæt Dronningen og den kongelige familie påhører, hvad regeringen har tænkt sig i den kommende samling. Det er min opgave som såkaldt aldersformand – dvs. ikke det ældste, men det længst siddende medlem – at åbne Tinget og få det sat, dvs. lede valget af Præsidium og tingsekretærer. Det lyder enkelt, men i Sverige kunne man jo se, at det er det ikke altid.  Til formand har Socialdemokratiet, Dansk Folkeparti, Venstre, Liberal Alliance, Det Konservative Folkeparti, Nunatta Qitornai, Tjóðveldi og Javnaðarflokkurin indstillet fru Pia Kjærsgaard. Der foreligger ikke andre indstillinger.  Til første næstformand har Socialdemokratiets

In [79]:
for i in range(5):
    print(len(what[1][i]))
    print(what[1][i])
    print()

96713
 Deres Majestæt, Deres Kongelige Højheder. I en tid, hvor demokratiet og folkestyrets institutioner i mange lande er under pres, er der al mulig grund til at højtideligholde folkestyrets festdag her i det danske Folketing. Den første tirsdag i oktober er en festdag, hvor folketingsåret efter gudstjenesten indledes med, at Folketinget i nærværelse af Hendes Majestæt Dronningen og den kongelige familie påhører, hvad regeringen har tænkt sig i den kommende samling. Det er min opgave som såkaldt aldersformand – dvs. ikke det ældste, men det længst siddende medlem – at åbne Tinget og få det sat, dvs. lede valget af Præsidium og tingsekretærer. Det lyder enkelt, men i Sverige kunne man jo se, at det er det ikke altid.  Til formand har Socialdemokratiet, Dansk Folkeparti, Venstre, Liberal Alliance, Det Konservative Folkeparti, Nunatta Qitornai, Tjóðveldi og Javnaðarflokkurin indstillet fru Pia Kjærsgaard. Der foreligger ikke andre indstillinger.  Til første næstformand har Socialdemokra

In [ ]:
df_agg_party_date = df.groupby(['GroupNameShort', df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x))

In [ ]:
df_agg_party_date.sort_index().head()

In [ ]:
df_agg_name = df.groupby(['FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x))
pd.DataFrame(df_agg_name).head()

In [ ]:
df_agg_date_party = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'GroupNameShort'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_party).head()

In [50]:
df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805840 entries, 0 to 805839
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   MeetingID        805840 non-null  int64         
 1   ItemNo           805840 non-null  int64         
 2   StartDateTime    805840 non-null  datetime64[ns]
 3   OratorFirstName  804225 non-null  object        
 4   OratorLastName   804225 non-null  object        
 5   GroupNameShort   678941 non-null  object        
 6   OratorRole       804225 non-null  object        
 7   TekstGruppe      805840 non-null  object        
 8   FullName         804225 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 55.3+ MB


In [ ]:
df_agg_date = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date).head()

In [ ]:
df_agg_date_name = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_name).head()

In [44]:
df = df[df['FullName'].notnull()]

,MeetingID,ItemNo,StartDateTime,OratorFirstName,OratorLastName,GroupNameShort,OratorRole,TekstGruppe,FullName,FulName
0,66,0,2019-02-27 13:01:45,Christian,Langballe,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...,Christian Langballe,Christian Langballe
1,66,0,2019-02-27 13:02:06,Søren Pape,Poulsen,None,minister,Tak for det. Og tak for spørgsmålet. Fremmedk...,Søren Pape Poulsen,Søren Pape Poulsen
2,66,0,2019-02-27 13:03:40,Christian,Langballe,DF,medlem,Det står tilbage at få besvaret nogle ting i ...,Christian Langballe,Christian Langballe
3,66,0,2019-02-27 13:04:55,Søren Pape,Poulsen,None,minister,"Tak. Jeg skal prøve, så godt jeg kan – det va...",Søren Pape Poulsen,Søren Pape Poulsen
4,66,0,2019-02-27 13:05:49,Christian,Langballe,DF,medlem,"Altså, jeg synes jo, at de her burkabrude, so...",Christian Langballe,Christian Langballe
...,...,...,...,...,...,...,...,...,...,...
805835,109,0,2020-05-14 10:02:35,Christoffer Aagaard,Melson,V,medlem,"Tak for det. Jeg tager ordet i dag, fordi vi ...",Christoffer Aagaard Melson,Christoffer Aagaard Melson
805836,109,1,2020-05-14 10:02:35,Christoffer Aagaard,Melson,V,medlem,"Tak for det. Jeg tager ordet i dag, fordi vi ...",Christoffer Aagaard Melson,Christoffer Aagaard Melson
805837,109,2,2020-05-14 10:02:35,Christoffer Aagaard,Melson,V,medlem,"Tak for det. Jeg tager ordet i dag, fordi vi ...",Christoffer Aagaard Melson,Christoffer Aagaard Melson
805838,109,3,2020-05-14 10:02:35,Christoffer Aagaard,Melson,V,medlem,"Tak for det. Jeg tager ordet i dag, fordi vi ...",Christoffer Aagaard Melson,Christoffer Aagaard Melson


In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df_agg_date[1]

## Topics

In [ ]:
stopwords = open('stopord.txt').read().splitlines() #Link: https://gist.github.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b

In [ ]:
# convert the text to a tf-idf weighted term-document matrix
from __future__ import division

# Hyperparameters
corpus = 700000 #Der er ca. 66247 forskellige ord, efter stopord
l1_ratio = 0
max_iter = 50000
no_components = 20 #number of topics

# The words should appear at least 'min_df' times and at most 'max_df' pr document. Decimals indicate proportions
min_df = 2
max_df = 0.6

data=df_agg_date_name

vectorizer = TfidfVectorizer(max_features=corpus, min_df=min_df, max_df=max_df, stop_words=stopwords)

X = vectorizer.fit_transform(data)
 
idx_to_word = np.array(vectorizer.get_feature_names())


# apply Non-negative Matrix Factorization to get topics using TF-IDF
nmf = NMF(n_components=no_components, solver="mu", l1_ratio = l1_ratio, max_iter = max_iter, random_state=1)

W = nmf.fit_transform(X)

H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))
    

In [ ]:
# List of words
idx_to_word

In [ ]:
# Last topic's most defining words
print(topic.argsort()[-10:])
print(idx_to_word[topic.argsort()[-10:]])

In [ ]:
#for i in range(len(W)):
#    print(W[i])

In [ ]:
#for i in range(len(H)):
#    print(H[i])

In [ ]:
print('There are ', len(data), ' rows in this aggregation')
print('and from these, ', W.shape[1], ' topics are computed.')
print()
print('There are ', len(idx_to_word),' words in the corpus.')
print(H.shape)

In [ ]:
idx_to_word[topic.argsort()[-10:]]

In [ ]:
pd.concat([data, W], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
#pd.reset_option('display.float_format')
pd.DataFrame(W)

In [ ]:
pd.DataFrame(H)

In [ ]:
# Write to a .csv file

#with open('topics_and_weights.csv', 'w', newline='', encoding='utf-16') as csvfile:
 #   topicwriter = csv.writer(csvfile, delimiter=',',
  #                          quotechar='"', quoting=csv.QUOTE_MINIMAL)
# Create the header
   # topicwriter.writerow(['Topic','Word','Weight'])

In [ ]:
# print the topics for tableau
 
#for i, topic in enumerate(H):
 
 #   max_weights = topic.argsort()[-10:]
  #  words = [str(x) for x in idx_to_word[topic.argsort()[-10:]]]
     
   # for count, item in enumerate(max_weights):
        
    #    with open('topics_and_weights.csv', 'a', newline='', encoding='utf-16') as csvfile:
     #       topicwriter = csv.writer(csvfile, delimiter=',',
      #                      quotechar='"', quoting=csv.QUOTE_MINIMAL)
       #     topicwriter.writerow(["Topic"+str(i+1) ,words[count],H[i,item]])

In [ ]:
#stopwords_edit = open('stopord.txt','a')
#stopwords_edit.write('\nenhedslisten')
#stopwords_edit.close()